In [1]:
# Widgets
import bqviz as bqv
from ipywidgets import *
from bqwidgets import *
from bqplot import *
import bqplot
from bqplot.interacts import LassoSelector
from scipy.optimize import minimize

# first step is to import the bql module
import bql
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os 
import warnings
warnings.filterwarnings("ignore")

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [2]:
index = 'ERL0 Index'
n_assets = 20

In [3]:
def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all

def cur_dur():
    dts = f.range('-10d', '0d')
    dur = d.duration(dates = dts, fill='prev')
    dur = f.last(f.dropna(f.if_(dur==0, np.nan, dur)))
    return f.if_(dur != np.nan, dur, 0)

def duration_contrib():
    ticker = d.id()
    issuer = d.ticker()
    aux_duration = cur_dur()
    duration = f.if_( aux_duration == np.nan, 0, aux_duration)
    weighted_dur = ticker['Weights']/100 * duration
    break_down = {'issuer duration contrib':f.sum(f.group(weighted_dur, issuer))}
    return break_down

#Equities Optimization functions
def EQ_f_obj(x):
    return (sum((np.matmul(A, x) - y)**2) + sum((np.matmul(Signal1, x) - y_signal1)**2) + sum((np.matmul(Signal2, x) - y_signal2)**2))**2

def EQ_constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

#Fixed Income Optimization functions
def FI_f_obj(x):
    return (sum((np.matmul(A, x) - y)**2) + sum((np.matmul(Signal1, x) - y_signal1)**2) + sum((np.matmul(Signal2, x) - y_signal2)**2) + sum((np.matmul(Signal3, x) - y_signal3)**2))**2

def FI_constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

def FI_constraint_total_duration_ub(x):  #The portfolio duration must be the same as the bechmark
    dur_bk = agg_type['Weights Type'].sum() 
    return dur_bk - np.sum(np.array(top_assets['Duration'].tolist())*x)

In [4]:
request =  bql.Request(index, {'idx_type': d.SECURITY_TYP()})
response = bq.execute(request)

if response[0].df().idx_type[0] == 'Equity Index':
    ID_INDEX_TYPE = 'EQUITY'
else:
    ID_INDEX_TYPE = 'FI'

In [5]:
#A lo de los members luego tendrás que meterle la fecha para el backtest

description = d.id()
weights = description['Weights']/100

if ID_INDEX_TYPE == 'EQUITY':
    
    #Index Aggregates ########
    
    #Signal 1
    sector = d.GICS_SECTOR_NAME()    
    group_sec = weights.group(sector).sum()
    
    request =  bql.Request(u.members(index), {'Weights Sector': group_sec['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    agg_sector = response[0].df().reset_index()
    agg_sector = agg_sector.rename(index=str, columns={'ID': 'Sector'})
    
    #Signal 2
    geo = d.CNTRY_OF_RISK()
    group_geo = weights.group(geo).sum()
    
    request =  bql.Request(u.members(index), {'Weights Geo': group_geo['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    agg_geo = response[0].df().reset_index()
    agg_geo = agg_geo.rename(index=str, columns={'ID': 'Geo'})
    
    
    #Find top assets based on weights of the corresponding signal aggrerate  ########

    #Signal 1
    group_sec = f.ungroup(weights.group(sector).sum())

    #Signal 2
    group_geo = f.ungroup(weights.group(geo).sum())

    #Get the weight of each signal and sort by the largest total weight
    total_weight = group_sec['value'] * group_geo['value'] * weights['value']

    request =  bql.Request(u.members(index), {'Total Weights': f.first(f.group(f.groupsort(total_weight['value'])), n_assets)}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    data = response[0].df().reset_index()

    request =  bql.Request(data['ORIG_IDS'].tolist(), {'Sector': sector, 'Geo': geo})
    response = bq.execute(request)

    top_assets = pd.merge(response[0].df().reset_index(), response[1].df().reset_index(), how = 'left', on = ['ID'])    
    
    
    #Optimization process #########
    
    request =  bql.Request(top_assets['ID'].tolist() + [index], {'Returns': f.dropna(d.DAY_TO_DAY_TOT_RETURN_GROSS_DVDS(START='-1M', END='0D'))}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    rt = response[0].df().reset_index()
    rt = rt.pivot(index='DATE', columns='ID', values='Returns').dropna()

    A = np.array(rt[top_assets['ID'].tolist()].values)
    y = np.array(rt[index].values)

    fil_agg_sector = agg_sector.loc[agg_sector['Sector'].isin(top_assets['Sector'].unique().tolist())]
    #Re-scale
    fil_agg_sector['Weights Sector'] = fil_agg_sector['Weights Sector']/sum(fil_agg_sector['Weights Sector'])

    fil_agg_geo = agg_geo.loc[agg_geo['Geo'].isin(top_assets['Geo'].unique().tolist())]
    #Re-scale
    fil_agg_geo['Weights Geo'] = fil_agg_geo['Weights Geo']/sum(fil_agg_geo['Weights Geo'])

    #Find same features of the signals in the asset list
    Signal1 = []
    for i_sec in fil_agg_sector['Sector'].unique().tolist():
        Signal1 += [top_assets['Sector'].isin([i_sec]).tolist()]

    Signal1 = np.array(Signal1)
    y_signal1 = np.array(fil_agg_sector['Weights Sector'].values)

    #Find same features of the signals in the asset list
    Signal2 = []
    for i_geo in fil_agg_geo['Geo'].unique().tolist():
        Signal2 += [top_assets['Geo'].isin([i_geo]).tolist()]

    Signal2 = np.array(Signal2)
    y_signal2 = np.array(fil_agg_geo['Weights Geo'].values)

    guess=np.zeros(len(top_assets['ID']))

    bnds = [(0,1) for i in range(len(top_assets['ID']))]
    results = minimize(fun = EQ_f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'maxiter':1000}, constraints= [{'type':'eq','fun' : EQ_constraint_total_weight_ub}])      
    
    
else:
    
    #Index Aggregates ########
    
    aux_duration = cur_dur()
    duration = f.if_( aux_duration == np.nan, 0, aux_duration)
    weighted_dur = weights * duration
    
    #Signal 1
    sector = d.BICS_LEVEL_1_SECTOR_NAME()    
    group_sec = f.sum(f.group(weighted_dur, sector))
    
    request =  bql.Request(u.members(index), {'Weights Sector': group_sec['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    agg_sector = response[0].df().reset_index()
    agg_sector = agg_sector.rename(index=str, columns={'ID': 'Sector'})
    
    #Signal 2
    geo = d.CNTRY_OF_RISK()
    group_geo = f.sum(f.group(weighted_dur, geo))
    
    request =  bql.Request(u.members(index), {'Weights Geo': group_geo['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    agg_geo = response[0].df().reset_index()
    agg_geo = agg_geo.rename(index=str, columns={'ID': 'Geo'})
    
    #Signal 3
    iss_type = d.PAYMENT_RANK()
    group_type = f.sum(f.group(weighted_dur, iss_type))
    
    request =  bql.Request(u.members(index), {'Weights Type': group_type['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    agg_type = response[0].df().reset_index()
    agg_type = agg_type.rename(index=str, columns={'ID': 'Iss_Type'})
    

    #Find top assets based on duration of the corresponding signal aggrerate  ########
    
    #Count the number of issuers
    ticker_name = d.ticker()

    group_ticker = f.group(ticker_name, ticker_name).count()
    group_ticker = group_ticker.group().count()

    request =  bql.Request(u.members(index), {'N_Issuers': group_ticker['value']}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    n_issuers = response[0].df().reset_index()
    n_issuers = n_issuers.loc[0, 'N_Issuers']

    #If the number of the required assets is greater than the number of issuers of the index,
    #then the algorithm will take the higher issue by contribution to duration of each isser.
    #If not, the selection is made by issues, not issuers. 
    if n_assets > n_issuers:   

        ticker_name = d.ticker()

        #Signal 1
        group_sec = f.ungroup(f.group(weighted_dur, sector).sum())

        #Signal 2
        group_geo = f.ungroup(f.group(weighted_dur, geo).sum())

        #Signal 3
        group_type = f.ungroup(f.group(weighted_dur, iss_type).sum())

        total_dur_issue = group_sec['value'] * group_geo['value'] * group_type['value'] * weighted_dur['value']

        filter_issue = u.filter(u.members(index), f.grouprank(total_dur_issue) <= n_assets) #<--------------------------------------------------- Fecha

        request =  bql.Request(filter_issue, {'Duration': duration['value'], 'Sector': sector['value'], 'Geo': geo['value'], 'Iss_Type': iss_type['value']})
        response = bq.execute(request)
        top_assets = bq_res_array_to_pd(response)

    else:

        ticker_name = d.ticker()

        #Signal 1        
        group_ticker = f.group(weighted_dur, ticker_name).sum()

        #Signal 2        
        group_sec = f.ungroup(f.group(weighted_dur, sector).sum())
        group_sec = f.group(group_sec, ticker_name).first()

        #Signal 3        
        group_geo = f.ungroup(f.group(weighted_dur, geo).sum())
        group_geo = f.group(group_geo, ticker_name).first()

        #Signal 4        
        group_type = f.ungroup(f.group(weighted_dur, iss_type).sum())
        group_type = f.group(group_type, ticker_name).first()

        total_dur_issuer = group_sec['value'] * group_geo['value'] * group_type['value'] * group_ticker['value']

        rank_total_dur_issuer = f.grouprank(total_dur_issuer)

        criteria1 = f.ungroup(rank_total_dur_issuer) <= n_assets
        criteria2 = weighted_dur == f.groupmax(weighted_dur, by = ticker_name)

        filter_issuers = u.filter(u.members(index), f.and_(criteria1, criteria2)) #<--------------------------------------------------- Fecha

        request =  bql.Request(filter_issuers, {'Duration': duration['value'], 'Sector': sector['value'], 'Geo': geo['value'], 'Iss_Type': iss_type['value']})
        response = bq.execute(request)
        top_assets = bq_res_array_to_pd(response)   

    #Optimization process #########

    request =  bql.Request(top_assets['ID'].tolist() + [index], {'Returns': f.dropna(d.DAY_TO_DAY_TOT_RETURN_GROSS_DVDS(START='-1M', END='0D'))}) #<--------------------------------------------------- Fecha
    response = bq.execute(request)
    rt = response[0].df().reset_index()
    rt = rt.pivot(index='DATE', columns='ID', values='Returns').dropna()

    A = np.array(rt[top_assets['ID'].tolist()].values)
    y = np.array(rt[index].values)

    fil_agg_sector = agg_sector.loc[agg_sector['Sector'].isin(top_assets['Sector'].unique().tolist())]
    #Re-scale
    fil_agg_sector['Weights Sector'] = fil_agg_sector['Weights Sector']/sum(fil_agg_sector['Weights Sector'])

    fil_agg_geo = agg_geo.loc[agg_geo['Geo'].isin(top_assets['Geo'].unique().tolist())]
    #Re-scale
    fil_agg_geo['Weights Geo'] = fil_agg_geo['Weights Geo']/sum(fil_agg_geo['Weights Geo'])

    fil_agg_type = agg_type.loc[agg_type['Iss_Type'].isin(top_assets['Iss_Type'].unique().tolist())]
    #Re-scale
    fil_agg_type['Weights Type'] = fil_agg_type['Weights Type']/sum(fil_agg_type['Weights Type'])

    #Find same features of the signals in the asset list
    Signal1 = []
    for i_sec in fil_agg_sector['Sector'].unique().tolist():
        Signal1 += [top_assets['Sector'].isin([i_sec]).tolist()]

    Signal1 = np.array(Signal1)
    y_signal1 = np.array(fil_agg_sector['Weights Sector'].values)

    #Find same features of the signals in the asset list
    Signal2 = []
    for i_geo in fil_agg_geo['Geo'].unique().tolist():
        Signal2 += [top_assets['Geo'].isin([i_geo]).tolist()]

    Signal2 = np.array(Signal2)
    y_signal2 = np.array(fil_agg_geo['Weights Geo'].values)    

    #Find same features of the signals in the asset list
    Signal3 = []
    for i_type in fil_agg_type['Iss_Type'].unique().tolist():
        Signal3 += [top_assets['Iss_Type'].isin([i_type]).tolist()]

    Signal3 = np.array(Signal3)
    y_signal3 = np.array(fil_agg_type['Weights Type'].values)

    guess=np.zeros(len(top_assets['ID']))

    bnds = [(0,1) for i in range(len(top_assets['ID']))]
    cons1 = {'type':'eq','fun' : FI_constraint_total_weight_ub}
    cons2 = {'type':'eq','fun' : FI_constraint_total_duration_ub}
    results = minimize(fun = FI_f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'maxiter':1000}, constraints= [cons1, cons2])

In [ ]:
end_date = datetime.today()
start_date = end_date - relativedelta(months = 12*5)

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    #Find top n issuers in the index by duration contribution
    
    idx = u.members(index, aux_date)
    req = bql.Request(idx, duration_contrib(aux_date))
    res = bq.execute(req)

    df = bq_res_array_to_pd(res)[['ID', 'issuer duration contrib', 'number issues']]
    df.columns = ['Issuer',aux_name, 'number issues'] 
    df = df.fillna(0)
    dur_index = sum(df[aux_name])    
    top_issuers = df.sort_values(by = [aux_name], ascending = False)
    top_issuers['rel cumsum number issues'] = top_issuers['number issues'].cumsum()/sum(top_issuers['number issues'])
    
    if len(top_issuers[top_issuers['rel cumsum number issues'] <= n_assets]) <= 1:
        top_issuers = pd.DataFrame(top_issuers.loc[top_issuers.index[0], :]).T
        one_issuer = True
    else:
        top_issuers = top_issuers[top_issuers['rel cumsum number issues'] <= n_assets]
        one_issuer = False
    list_issuers = top_issuers['Issuer'].tolist()  
    
    #Print the issues of the top issuers    
    criteria1 = d.ticker() == ', '.join(list_issuers)
    filtered_univ = u.filter(idx, criteria1)         

    ticker = d.id()
    aux_duration = cur_dur(aux_date)['value'] 
    num_matur = (d.maturity() - f.todate(aux_date))/365
    duration = f.if_( aux_duration == np.nan, num_matur, aux_duration)
    weighted_dur = ticker['Weights']/100 * duration
    rank = d.PAYMENT_RANK()
    flds = {'Issuer': d.ticker(), 'Issue Contribution': weighted_dur, 'Duration': duration, 'Rank': rank}
    request =  bql.Request(filtered_univ, flds)
    res = bq.execute(request)
    df = bq_res_array_to_pd(res)

    result = pd.merge(df, top_issuers, on = 'Issuer')
    result = result.sort_values(by = [aux_name, 'Issue Contribution'], ascending = False)
    result = result[result.Rank != 'Secured']    
    
    if one_issuer == True:
        result['rel cumsum number issues'] = result['rel cumsum number issues'].cumsum()/result['number issues'][0]
        result = result[result['rel cumsum number issues'] <= n_assets]    
    
#     #Get Index Duration
#     ticker = d.id()
#     aux_duration = cur_dur(aux_date)
#     num_matur = (d.maturity() - f.todate(aux_date))/365
#     duration = f.if_( aux_duration == np.nan, num_matur, aux_duration)
#     weighted_dur = ticker['Weights']/100 * duration

#     req = bql.Request(idx, {'idx duration': weighted_dur.group().sum()})
#     res = bq.execute(req)
#     dur_index = res[0].df().reset_index()['idx duration'][0]

    dur_port = sum(result['Issue Contribution'])
    result['Issue Contribution'] = result['Issue Contribution']*(dur_index/dur_port)    

    asset_list = []
    weights1=[]
    for issuer in list_issuers :
        try :
            iss_list = result.loc[result.loc[:,'Issuer'] == issuer, 'ID' ].tolist()
            contri_list = result.loc[result.loc[:,'Issuer'] == issuer, 'Issue Contribution' ].tolist()
            dur_list = result.loc[result.loc[:,'Issuer'] == issuer, 'Duration' ].tolist()

            weights1 += list(np.array(contri_list)/np.array(dur_list))
            asset_list += iss_list 
        except :
            continue

    weights = np.array(weights1)
    weights = weights / sum(weights)
    
    if ucits == True:
        weights = UCITS_Weights(weights)
          
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': asset_list, 'weight': weights})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': asset_list, 'weight': weights})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)

In [8]:
description = d.id()
weights = description['Weights']/100

#Index Aggregates ########

aux_duration = cur_dur()
duration = f.if_( aux_duration == np.nan, 0, aux_duration)
weighted_dur = weights * duration

#Signal 1
sector = d.BICS_LEVEL_1_SECTOR_NAME()    
group_sec = f.sum(f.group(weighted_dur, sector))

request =  bql.Request(u.members(index), {'Weights Sector': group_sec['value']}) #<--------------------------------------------------- Fecha
response = bq.execute(request)
agg_sector = response[0].df().reset_index()
agg_sector = agg_sector.rename(index=str, columns={'ID': 'Sector'})

#Signal 2
geo = d.CNTRY_OF_RISK()
group_geo = f.sum(f.group(weighted_dur, geo))

request =  bql.Request(u.members(index), {'Weights Geo': group_geo['value']}) #<--------------------------------------------------- Fecha
response = bq.execute(request)
agg_geo = response[0].df().reset_index()
agg_geo = agg_geo.rename(index=str, columns={'ID': 'Geo'})

#Signal 3
iss_type = d.PAYMENT_RANK()
group_type = f.sum(f.group(weighted_dur, iss_type))

request =  bql.Request(u.members(index), {'Weights Type': group_type['value']}) #<--------------------------------------------------- Fecha
response = bq.execute(request)
agg_type = response[0].df().reset_index()
agg_type = agg_type.rename(index=str, columns={'ID': 'Iss_Type'})


#Find top assets based on duration of the corresponding signal aggrerate  ########

#Count the number of issuers
ticker_name = d.ticker()

group_ticker = f.group(ticker_name, ticker_name).count()
group_ticker = group_ticker.group().count()

request =  bql.Request(u.members(index), {'N_Issuers': group_ticker['value']}) #<--------------------------------------------------- Fecha
response = bq.execute(request)
n_issuers = response[0].df().reset_index()
n_issuers = n_issuers.loc[0, 'N_Issuers']

In [9]:
ticker_name = d.ticker()

#Signal 1        
group_ticker = f.group(weighted_dur, ticker_name).sum()

#Signal 2        
group_sec = f.ungroup(f.group(weighted_dur, sector).sum())
group_sec = f.group(group_sec, ticker_name).first()

#Signal 3        
group_geo = f.ungroup(f.group(weighted_dur, geo).sum())
group_geo = f.group(group_geo, ticker_name).first()

#Signal 4
group_type = f.ungroup(f.group(weighted_dur, iss_type).sum())
group_type = f.group(group_type, ticker_name).first()

total_dur_issuer = group_sec['value'] * group_geo['value'] * group_type['value'] * group_ticker['value']

rank_total_dur_issuer = f.grouprank(total_dur_issuer) #<------------------------------------------------------------------ ESTO LO ORDENA???????????????????????????????????????????????

criteria1 = f.ungroup(rank_total_dur_issuer) <= n_assets
criteria2 = weighted_dur == f.groupmax(weighted_dur, by = ticker_name)

filter_issuers = u.filter(u.members(index), f.and_(criteria1, criteria2)) #<--------------------------------------------------- Fecha

request =  bql.Request(filter_issuers, {'Duration': duration['value'], 'Sector': sector['value'], 'Geo': geo['value'], 'Iss_Type': iss_type['value']})
response = bq.execute(request)
top_assets = bq_res_array_to_pd(response)
top_assets

ID Geo          Iss_Type                  Sector   Duration
0   AX917386 Corp  FR      Sr Unsecured              Financials   6.744971
1   ZS409525 Corp  US      Sr Unsecured              Financials   9.556236
2   EI231370 Corp  FR      Sr Unsecured               Utilities   9.074487
3   AN646290 Corp  FR      Sr Unsecured              Financials  15.602175
4   LW953171 Corp  US      Sr Unsecured              Financials   6.611504
5   AR547530 Corp  US      Sr Unsecured              Financials  13.841942
6   AR776212 Corp  DE      Sr Unsecured              Financials  15.332543
7   QZ019053 Corp  US      Sr Unsecured              Financials   7.294119
8   AT634853 Corp  US      Sr Unsecured              Financials   5.781074
9   JK446099 Corp  DE      Sr Unsecured          Communications   8.262929
10  AW746868 Corp  FR  Sr Non Preferred              Financials   6.089056
11  EK795322 Corp  FR      Subordinated              Financials   7.023527
12  AO968337 Corp  GB      Subordinated              Financials   7.760516
13  ZS804165 Corp  FR      Sr Unsecured              Financials  10.686062
14  AO113936 Corp  DE      Sr Unsecured  Consumer Discretionary  15.568285
15  EK488426 Corp  FR      Sr Unsecured                  Energy   9.279433
16  AN549665 Corp  US      Sr Unsecured             Industrials  13.262226
17  AN575183 Corp  US      Sr Unsecured              Financials   7.382213
18  AN318761 Corp  US      Sr Unsecured              Financials   7.316421
19  AX003584 Corp  DE      Sr Unsecured  Consumer Discretionary   8.993924

In [11]:
request =  bql.Request(filter_issuers, {'Rank': f.ungroup(rank_total_dur_issuer)['value']})
response = bq.execute(request)
top_assets2 = bq_res_array_to_pd(response)
top_assets2

ID  Rank
0   AX917386 Corp   8.0
1   ZS409525 Corp   6.0
2   EI231370 Corp  15.0
3   AN646290 Corp   9.0
4   LW953171 Corp   5.0
5   AR547530 Corp   4.0
6   AR776212 Corp  10.0
7   QZ019053 Corp   3.0
8   AT634853 Corp   7.0
9   JK446099 Corp  19.0
10  AW746868 Corp  18.0
11  EK795322 Corp  16.0
12  AO968337 Corp  20.0
13  ZS804165 Corp  11.0
14  AO113936 Corp  13.0
15  EK488426 Corp  17.0
16  AN549665 Corp  12.0
17  AN575183 Corp   2.0
18  AN318761 Corp   1.0
19  AX003584 Corp  14.0

In [7]:
#PRUEBA ARBOL <----------------- ESTO FUNCIONA!!!!!!!!!!!!!!!!!!!!!!!!!!!!

ticker_name = d.ticker()

description = d.id()
weights = description['Weights']/100

#Index Aggregates ########

aux_duration = cur_dur()
duration = f.if_( aux_duration == np.nan, 0, aux_duration)
weighted_dur = weights * duration
sector = d.BICS_LEVEL_1_SECTOR_NAME()    
geo = d.CNTRY_OF_RISK()
iss_type = d.PAYMENT_RANK()

#Signal 1        
group_ticker = f.group(weighted_dur, by=[ticker_name, iss_type, sector, geo]).sum()

request =  bql.Request(u.members(index), {'Duration': group_ticker})
response = bq.execute(request)
response[0].df()

ORIG_IDS TICKER()  \
ID                                                                      
AALLN:Sr Unsecured:Materials:ZA                         None    AALLN   
AAPL:Sr Unsecured:Technology:US                         None     AAPL   
ABBNVX:Sr Unsecured:Industrials:CH                      None   ABBNVX   
ABBV:Sr Unsecured:Health Care:US                        None     ABBV   
ABESM:Sr Unsecured:Industrials:ES                       None    ABESM   
ABIBB:Sr Unsecured:Consumer Staples:BE                  None    ABIBB   
ABNANV:Sr Preferred:Financials:NL                       None   ABNANV   
ABNANV:Subordinated:Financials:NL                       None   ABNANV   
ABT:Sr Unsecured:Health Care:US                         None      ABT   
ACAFP:Sr Non Preferred:Financials:FR                    None    ACAFP   
ACAFP:Sr Preferred:Financials:FR                        None    ACAFP   
ACAFP:Sr Unsecured:Financials:FR               AZ247560 Corp    ACAFP   
ACAFP:Subordinated:Financials:FR                        None    ACAFP   
ACEIM:Sr Unsecured:Utilities:IT                         None    ACEIM   
ACFP:Sr Unsecured:Consumer Discretionary:FR             None     ACFP   
ACHMEA:Jr Subordinated:Health Care:NL          EK723977 Corp   ACHMEA   
ACHMEA:Sr Preferred:Financials:NL                       None   ACHMEA   
ACHMEA:Sr Unsecured:Health Care:NL             EJ907063 Corp   ACHMEA   
ACHMEA:Subordinated:Health Care:NL             EJ595440 Corp   ACHMEA   
ACQUIU:Sr Unsecured:Utilities:IT               AX048246 Corp   ACQUIU   
ACSSCE:Sr Unsecured:Industrials:ES             AS087411 Corp   ACSSCE   
ADENVX:Sr Unsecured:Consumer Discretionary:CH  AL417646 Corp   ADENVX   
ADM:Sr Unsecured:Consumer Staples:US                    None      ADM   
ADNA:Sr Unsecured:Consumer Staples:NL                   None     ADNA   
ADPFP:Sr Unsecured:Industrials:FR                       None    ADPFP   
ADRIT:Sr Unsecured:Industrials:IT              AN738727 Corp    ADRIT   
AEGON:Sr Non Preferred:Financials:NL           AZ170186 Corp    AEGON   
AEGON:Sr Unsecured:Financials:NL               AL502810 Corp    AEGON   
AEGON:Subordinated:Financials:NL               EK179322 Corp    AEGON   
AEMSPA:Sr Unsecured:Utilities:IT               EJ981434 Corp   AEMSPA   
...                                                      ...      ...   
VOD:Jr Subordinated:Communications:GB                   None      VOD   
VOD:Sr Unsecured:Communications:GB                      None      VOD   
VPARKI:Sr Unsecured:Consumer Discretionary:FR           None   VPARKI   
VVOYHT:Sr Unsecured:Financials:FI                       None   VVOYHT   
VW:Jr Subordinated:Consumer Discretionary:DE            None       VW   
VW:Sr Non Preferred:Consumer Discretionary:DE           None       VW   
VW:Sr Unsecured:Consumer Discretionary:DE               None       VW   
VZ:Sr Unsecured:Communications:US                       None       VZ   
WBA:Sr Unsecured:Consumer Staples:US           EK594401 Corp      WBA   
WESAU:Sr Unsecured:Consumer Staples:AU                  None    WESAU   
WFC:Sr Unsecured:Financials:US                          None      WFC   
WHR:Sr Unsecured:Consumer Discretionary:US              None      WHR   
WKLNA:Sr Unsecured:Technology:NL                        None    WKLNA   
WLTW:Sr Unsecured:Financials:US                LW004262 Corp     WLTW   
WMT:Sr Unsecured:Consumer Staples:US                    None      WMT   
WPC:Sr Preferred:Financials:US                          None      WPC   
WPC:Sr Unsecured:Financials:US                 EK699140 Corp      WPC   
WPPLN:Sr Unsecured:Communications:GB                    None    WPPLN   
WSTP:Sr Unsecured:Financials:AU                         None     WSTP   
WSTP:Sr Unsecured:Financials:NZ                AZ207709 Corp     WSTP   
WURTH:Sr Unsecured:Industrials:DE                       None    WURTH   
XL:Subordinated:Financials:BM                  AO083869 Corp       XL   
XYL:Sr Unsecured:Industrials:US      

In [8]:
response[0].df().to_excel('prueba_arbol.xlsx')

In [9]:
sector = d.BICS_LEVEL_1_SECTOR_NAME()    
group_sec = f.sum(f.group(weighted_dur, sector))

request =  bql.Request(u.members(index), {'Weights Sector': group_sec['value']}) #<--------------------------------------------------- Fecha
response = bq.execute(request)
agg_sector = response[0].df().reset_index()
agg_sector['Weights Sector'].sum()

3.9207445318817933